In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange

sys.path.append("../../")
import biked_commons
from biked_commons.design_evaluation.design_evaluation import get_standard_evaluations
from biked_commons.resource_utils import split_datasets_path
from biked_commons.conditioning import conditioning

from biked_commons.design_evaluation.scoring import *
from biked_commons.benchmark_models import benchmarking_utils
from biked_commons.transformation.one_hot_encoding import encode_to_continuous, decode_to_mixed

c:\Users\Lyler\mambaforge\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(split_datasets_path("bike_bench.csv"), index_col=0)

device = "cpu"

In [3]:
def get_condition(idx=0):
    rider_condition = conditioning.sample_riders(10, split="test")
    use_case_condition = conditioning.sample_use_case(10, split="test")
    image_embeddings = conditioning.sample_image_embedding(10, split="test")
    condition = {"Rider": rider_condition[idx], "Use Case": use_case_condition[idx], "Embedding": image_embeddings[idx]}
    return condition


In [4]:
# from pymoo.core.problem import Problem
# from pymoo.algorithms.moo.nsga2 import NSGA2
# from pymoo.optimize import minimize
# class BikeBenchProblem(Problem):
#     def __init__(self, data_sample_df, conditioning):
#         evaluator, requirement_names, requirement_types = construct_tensor_evaluator(get_standard_evaluations("cpu"), data_sample_df.columns)
#         data_sample = data_sample_df.to_numpy()
        
#         self.conditioning = conditioning

#         self.evaluator=evaluator
#         self.requirement_names=requirement_names
#         isobjective = torch.tensor(requirement_types) == 1
#         self.isobjective = isobjective

#         n_var = data_sample.shape[1]
#         n_obj = torch.sum(isobjective)
#         n_ieq_constr = torch.sum(~isobjective)
#         xl = np.quantile(data_sample, 0.01, axis=0)
#         xu = np.quantile(data_sample, 0.99, axis=0)
#         super().__init__(n_var=n_var, n_obj=n_obj, n_ieq_constr=n_ieq_constr, xl=xl, xu=xu)

#     def evaluate_fn(self, X):
#         X_tens = torch.tensor(X, dtype=torch.float32)
#         eval_scores = self.evaluator(X_tens, self.conditioning)
#         objective_scores = eval_scores[:, self.isobjective].detach().numpy()
#         constraint_scores = eval_scores[:, ~self.isobjective].detach().numpy()
#         return objective_scores, constraint_scores

#     def _evaluate(self, x, out, *args, **kwargs):
#         objective_scores, constraint_scores = self.evaluate_fn(x)
#         out["F"] = objective_scores
#         out["G"] = constraint_scores

In [5]:
import numpy as np
import pandas as pd
import torch

from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Choice, Binary


class BikeBenchProblem(ElementwiseProblem):
    def __init__(self,
                 data_sample_df: pd.DataFrame,   # your one-hot–encoded df
                 conditioning: dict,
                 **kwargs):
        # 1) keep around the original continuous columns for the evaluator:
        self.continuous_cols = list(data_sample_df.columns)

        # 2) build your evaluator as before:
        evaluator, requirement_names, requirement_types = construct_tensor_evaluator(
            get_standard_evaluations("cpu"),
            self.continuous_cols
        )
        self.evaluator      = evaluator
        is_obj              = torch.tensor(requirement_types) == 1
        self.isobjective    = is_obj
        n_obj               = int(is_obj.sum().item())
        n_ieq_constr        = int((~is_obj).sum().item())
        self.conditioning   = conditioning

        # 3) decode the ONE-HOT df into a mixed-type df for variable inference:
        mixed_df = decode_to_mixed(data_sample_df)

        # 4) infer each variable from its mixed dtype
        vars = {}
        for col in mixed_df.columns:
            series = mixed_df[col]
            if pd.api.types.is_bool_dtype(series):
                vars[col] = Binary()
            elif pd.api.types.is_integer_dtype(series):
                low, high    = int(series.min()), int(series.max())
                vars[col]    = Integer(bounds=(low, high))
            elif pd.api.types.is_float_dtype(series):
                # use your desired quantiles for bounds
                low, high    = np.quantile(series, 0.01), np.quantile(series, 0.99)
                vars[col]    = Real(bounds=(float(low), float(high)))
            else:
                # categorical / object
                opts             = series.dropna().unique().tolist()
                vars[col]        = Choice(options=opts)
        super().__init__(
            vars=vars,
            n_obj=n_obj,
            n_ieq_constr=n_ieq_constr,
            **kwargs                                 
        )

    def _evaluate(self, X, out, *args, **kwargs):
        # X is a dict: { col_name: mixed_value }
        # 1) build a one-row DataFrame in the mixed space
        mixed_row = pd.DataFrame([X], columns=self.vars.keys())

        # 2) encode it back to continuous one-hot form
        cont_row  = encode_to_continuous(mixed_row)

        # 3) ensure the columns line up exactly with the original one-hot df
        cont_row  = cont_row[self.continuous_cols]

        # 4) to numpy → tensor
        x_np      = cont_row.to_numpy().astype(np.float32)
        x_t       = torch.tensor(x_np, dtype=torch.float32)

        # 5) evaluate
        scores    = self.evaluator(x_t, self.conditioning)

        # 6) split objectives vs constraints
        f = scores[:,   self.isobjective ].detach().numpy().flatten().tolist()   # ← as list
        g = scores[:, (~self.isobjective) ].detach().numpy().flatten().tolist()

        out["F"] = f
        out["G"] = g


In [21]:
from pymoo.core.mixed import MixedVariableGA
from pymoo.algorithms.moo.nsga2 import RankAndCrowdingSurvival
from pymoo.optimize import minimize

for i in range(1):
    condition = get_condition(i)
    problem = BikeBenchProblem(data, condition)

    algorithm = MixedVariableGA(pop_size=100, survival = RankAndCrowdingSurvival())

    res = minimize(problem, algorithm, ('n_gen', 10), seed=1, verbose=True)
    mixed_df = pd.DataFrame(list(res.X)  , columns=problem.vars.keys())
    res_df_onehot = encode_to_continuous(mixed_df)
    results_tens = torch.tensor(res_df_onehot.values, dtype=torch.float32)
    benchmarking_utils.evaluate_uncond(results_tens, "NSGA2", i, data.columns, device="cpu")

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.000000E+00 |  1.444215E+02 |  0.4683860011 |  0.6532621384
     2 |      200 |  0.000000E+00 |  2.262144E+01 |  0.4825900688 |  0.6532621384
     3 |      300 |  0.000000E+00 |  0.6577757210 |  0.4568853946 |  0.6532621384
     4 |      400 |  0.000000E+00 |  0.1347773373 |  0.4491379036 |  0.6532621384
     5 |      500 |  0.000000E+00 |  0.000000E+00 |  0.4357045433 |  0.6532621384
     6 |      600 |  0.000000E+00 |  0.000000E+00 |  0.4316216618 |  0.3342646360
     7 |      700 |  0.000000E+00 |  0.000000E+00 |  0.4284055734 |  0.5461795926
     8 |      800 |  0.000000E+00 |  0.000000E+00 |  0.4173671034 |  0.2564312518
     9 |      900 |  0.000000E+00 |  0.000000E+00 |  0.4098626865 |  0.5983542204
    10 |     1000 |  0.000000E+00 |  0.000000E+00 |  0.4175667900 |  0.3933217525


In [22]:
# result_df = pd.DataFrame(result_tens.numpy(), columns=data.columns)

In [23]:
# from biked_commons.api import rendering
# renderer = rendering.RenderingEngine(number_rendering_servers = 1, server_init_timeout_seconds=120)

# res1 = result_df.iloc[0]
# res = renderer.render_clip(res1)
# svg = res.image_bytes
# from IPython.display import SVG, display
# display(SVG(svg))